In [ ]:
# imports
import sys
sys.path.append('../')
import math, statistics, time
from collections import defaultdict
import numpy as np
from datetime import datetime
import pickle
import pandas as pd
import torch.nn as nn
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, InputExample
from tqdm import tqdm
from utils.sbert_meme_classifier import Classifier

base_model = 'roberta-base'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

with open('../data/training_label_100.pkl', 'rb') as f:
    labels = pickle.load(f)

In [2]:
# load meme dataset
meme_dict = None
with open('../data/meme_900k_cleaned_data_v2.pkl', 'rb') as f:
    meme_dict = pickle.load(f)
print("Keys in meme dict dataset:", meme_dict.keys())
print("Number of uuids:", len(meme_dict['uuid_label_dic']))

Keys in meme dict dataset: dict_keys(['label_uuid_dic', 'uuid_label_dic', 'uuid_caption_dic', 'uuid_image_path_dic', 'uuid_caption_cased_dic'])
Number of uuids: 300


In [3]:
# utility functions
def clean_and_unify_caption(caption):
    return caption[0].strip()+'; '+caption[1].strip()

In [4]:
# create pandas dataframe
training_uuids = labels.keys()
temp_arr = []
for uuid in training_uuids:
    for caption in meme_dict['uuid_caption_dic'][uuid]:
        temp_arr.append([uuid, clean_and_unify_caption(caption)])
df = pd.DataFrame(temp_arr, columns=['category', 'text'])

# split dataset
np.random.seed(42)
df_train, df_test = np.split(df.sample(frac=1, random_state=42), [int(.9*len(df))])

print(len(df_train), len(df_test))

270000 30000


### Analysis for sbert models

In [5]:
ks = [1,3,5,10]
model_names = ['sentence_transformer_roberta_samples_100_epochs_{}'.format(i*5) for i in range(1, 6)]

In [6]:
accuracy_dict = {}

In [8]:
# with open('accuracy_dict.pkl', 'wb') as f:
#     pickle.dump(accuracy_dict, f)

### Analysis for short sbert

In [5]:
ks = [1,3,5,10]
model_names = ['sentence_transformer_roberta_samples_100_epochs_{}'.format(i*5) for i in range(1,6)]

In [6]:
accuracy_dict = {}

In [7]:
for model_name in model_names:
    clf = Classifier(model_name=model_name, labels_path='../data/training_label_100.pkl')
    accuracy_dict[(model_name, 'train')] = clf.topKAccuracy(ks, df_train)
    accuracy_dict[(model_name, 'test')] = clf.topKAccuracy(ks, df_test)

  0%|          | 1/528 [00:16<2:25:55, 16.61s/it]


KeyboardInterrupt: 

In [10]:
accuracy_dict

{'sentence_transformer_num_classes_24_num_epochs_5': {1: 0.6145833333333334,
  3: 0.7684722222222222,
  5: 0.8393055555555555,
  10: 0.9293055555555556}}